In [0]:
!pip install Pillow
!pip install PIL
!pip install image

  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
#helps Colab install torch

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
#import the required packages

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from google.colab import drive
drive.mount('/content/drive')

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CUDA is available!  Training on GPU ...


In [0]:
#!wget "https://s3.amazonaws.com/video.udacity-data.com/topher/2018/September/5baa60a0_flower-photos/flower-photos.zip" -P "drive/My Drive/Colab_Notebooks/pytorch_challenge"
#!unzip "drive/My Drive/Colab_Notebooks/pytorch_challenge/flower-photos.zip" -d "drive/My Drive/Colab_Notebooks/pytorch_challenge/flower"

In [0]:
#!ls "drive/My Drive/Colab_Notebooks/pytorch_challenge"

In [0]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

data_dir = 'drive/My Drive/Colab_Notebooks/pytorch_challenge/new_flower_classification_model/photos/flower_data'

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 25
# percentage of training set to use as validation
valid_size = 0.25

# convert data to a normalized torch.FloatTensor
train_transforms = transforms.Compose([transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
                                      ])
                                       
test_transforms = transforms.Compose([
        transforms.Resize(224 + 32),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/valid', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
# choose the training and test datasets

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = [n for n in range(1,103)]

In [0]:
#!ls '/content/drive/My Drive/Colab_Notebooks'

#cwd = os.getcwd()
#print(cwd)

In [0]:
"""
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(50, 8))
# display 20 images
for idx in np.arange(10):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])
    """

'\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\n# helper function to un-normalize and display an image\ndef imshow(img):\n    img = img / 2 + 0.5  # unnormalize\n    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image\n    \n# obtain one batch of training images\ndataiter = iter(train_loader)\nimages, labels = dataiter.next()\nimages = images.numpy() # convert images to numpy for display\n\n# plot the images in the batch, along with the corresponding labels\nfig = plt.figure(figsize=(50, 8))\n# display 20 images\nfor idx in np.arange(10):\n    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])\n    imshow(images[idx])\n    ax.set_title(classes[labels[idx]])\n    '

In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet152(pretrained=True)
input_features = model.fc.in_features

for param in model.parameters():
    param.requires_grad = False


for i , param in enumerate(model.parameters()):
    if(i>300):
      param.requires_grad=True

print(input_features)
#creating our own classifier
classifier = nn.Sequential(nn.Linear(input_features, 512),
                            nn.ReLU(),
                            #nn.Dropout(0.3),
                            nn.Linear(512, 102),
                            nn.LogSoftmax(dim=1))
     
model.fc = classifier
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

epoch = 1

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:13<00:00, 18574049.45it/s]


2048


In [0]:

#To Continue Training or To use the saved model (resnet_keep_training)
#model = resnet152(*args, **kwargs)
#optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load('drive/My Drive/Colab_Notebooks/pytorch_challenge/flower_classification_models/resnet152_keep_training.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

#model.eval()
# - or -
model.train()
model = model.to(device)


In [0]:
# number of epochs to train the model
n_epochs = 100
#checkpoint_path = "flower_classification_models/resnet_18.pt"

validation_loss = []
training_loss = []

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(epoch, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    exp_lr_scheduler.step()
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    validation_loss.append(valid_loss)
    training_loss.append(train_loss)
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('lr = {:.6f} Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(optimizer.param_groups[0]['lr'], valid_loss_min,valid_loss))
        torch.save(model.state_dict(), 'drive/My Drive/Colab_Notebooks/pytorch_challenge/flower_classification_models/resnet152.pt')
        valid_loss_min = valid_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'drive/My Drive/Colab_Notebooks/pytorch_challenge/flower_classification_models/resnet152_keep_training.pt')

KeyboardInterrupt: ignored

In [0]:
#load the last saved model
#model.load_state_dict(torch.load('drive/My Drive/Colab_Notebooks/pytorch_challenge/flower_classification_models/resnet152.pt'))
checkpoint = torch.load('drive/My Drive/Colab_Notebooks/pytorch_challenge/flower_classification_models/resnet152_keep_training.pt')
model.load_state_dict(checkpoint['model_state_dict'])
#'drive/My Drive/Colab_Notebooks/pytorch_challenge/flower_classification_models/resnet152_keep_training.pt'


# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(18):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(102):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.075710

Test Accuracy of     1: 100% ( 8/ 8)
Test Accuracy of     2: 100% ( 4/ 4)
Test Accuracy of     3: 100% ( 6/ 6)
Test Accuracy of     4: N/A (no training examples)
Test Accuracy of     5: 100% ( 4/ 4)
Test Accuracy of     6: 100% (10/10)
Test Accuracy of     7: 100% ( 4/ 4)
Test Accuracy of     8: N/A (no training examples)
Test Accuracy of     9: N/A (no training examples)
Test Accuracy of    10: 100% ( 7/ 7)
Test Accuracy of    11: 100% ( 2/ 2)
Test Accuracy of    12: 100% ( 9/ 9)
Test Accuracy of    13: 100% (11/11)
Test Accuracy of    14: 100% ( 4/ 4)
Test Accuracy of    15: 100% ( 3/ 3)
Test Accuracy of    16: 100% ( 3/ 3)
Test Accuracy of    17: 100% ( 4/ 4)
Test Accuracy of    18: 100% ( 8/ 8)
Test Accuracy of    19: 100% ( 5/ 5)
Test Accuracy of    20: 100% ( 5/ 5)
Test Accuracy of    21: 100% ( 2/ 2)
Test Accuracy of    22: 100% ( 3/ 3)
Test Accuracy of    23: 100% ( 1/ 1)
Test Accuracy of    24: 100% ( 5/ 5)
Test Accuracy of    25: N/A (no training examples

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.plot(training_loss, label='Training loss')
plt.plot(validation_loss, label='validation_loss')
plt.legend(frameon=False)